
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y4", label="solidos")

/tmp/ipykernel_3858/178310800.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-20 22:53:58.702918: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 22:54:01.109436: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 22:54:01.109512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 22:54:01.458018: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 22:54:02.219229: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-20 22:54:02.220576: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.5,
            hidden_sizes = [[25, 10], [30, 15], [45, 20]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.02])

Testando combinacao1: Hidden Size=[25, 10], regularizer=0.02, learning_rate=0.02
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  3.0434498659999463
Stopped at epoch:  75
1/1 [==============================] - 0s 15ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.345164762999957
Stopped at epoch:  11
1/1 [==============================] - 0s 18ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.22772487600002478
Stopped at epoch:  7
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.9406674310000653
Stopped at epoch:  30
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.44412225400003535
Stopped at epoch:  12
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.6460451340000191
Stopped at epoch:  19
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 +++

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_5_0,0.504081,0.203005,-0.035938,0.228363,59.808010,63.344448,50.590786,57.342529
model_2_5_2,0.505222,0.197193,-0.040117,0.223776,59.670422,63.806381,50.794865,57.683361
model_2_5_1,0.505485,0.207328,-0.038916,0.229887,59.638653,63.000889,50.736195,57.229286
model_2_5_3,0.506319,0.201889,-0.042666,0.225645,59.538105,63.433201,50.919319,57.544514
model_2_5_4,0.515737,0.198583,-0.050936,0.221217,58.402256,63.695976,51.323223,57.873520
model_2_5_5,0.527685,0.248088,-0.060147,0.246400,56.961388,59.761307,51.773052,56.002144
model_2_5_6,0.530734,0.231712,-0.077332,0.231814,56.593620,61.062847,52.612297,57.086094
model_2_5_7,0.530841,0.231262,-0.081376,0.230308,56.580681,61.098640,52.809776,57.198006
model_2_5_8,0.537040,0.232989,-0.092304,0.227906,55.833076,60.961384,53.343449,57.376503
model_2_5_9,0.545561,0.240711,-0.096761,0.230900,54.805492,60.347603,53.561104,57.153976


DataFrame salvo em ../results/metrics_2_5
+++++++++++ [2_6] | 1 ++++++++++++++++++
Elapsed time:  0.3513162709999733
Stopped at epoch:  9
1/1 [==============================] - 0s 16ms/step
+++++++++++ [2_6] | 2 ++++++++++++++++++
Elapsed time:  0.2580568120000635
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [2_6] | 3 ++++++++++++++++++
Elapsed time:  0.2449339270000337
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_6] | 4 ++++++++++++++++++
Elapsed time:  0.24289540400002352
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [2_6] | 5 ++++++++++++++++++
Elapsed time:  0.2442700579999837
Stopped at epoch:  6
1/1 [==============================] - 0s 20ms/step
+++++++++++ [2_6] | 6 ++++++++++++++++++
Elapsed time:  0.289132726000048
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_6] | 7 ++++++++++++++++++
Elapsed time:  0.376816806000

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_2_6_22,0.506081,0.519304,-0.705709,-0.090789,59.566780,55.294334,247.638489,145.809219
model_2_6_21,0.507259,0.520882,-0.698755,-0.086515,59.424702,55.112846,246.628784,145.237930
model_2_6_2,0.507500,0.527836,-0.515155,0.010491,59.395592,54.312962,219.973328,132.270813
model_2_6_20,0.508564,0.523669,-0.693027,-0.082318,59.267357,54.792240,245.797180,144.676926
model_2_6_19,0.510609,0.526927,-0.678080,-0.073195,59.020725,54.417511,243.627121,143.457397
model_2_6_1,0.511198,0.687259,-0.451116,0.115851,58.949715,35.974545,210.676117,118.187019
model_2_6_18,0.511466,0.530586,-0.673537,-0.069206,58.917358,53.996628,242.967682,142.924194
model_2_6_17,0.511792,0.529275,-0.669014,-0.067491,58.878044,54.147388,242.310974,142.694962
model_2_6_16,0.512097,0.527006,-0.665015,-0.066482,58.841198,54.408401,241.730408,142.560043
model_2_6_13,0.512868,0.528818,-0.658171,-0.062157,58.748283,54.199936,240.736786,141.981949


DataFrame salvo em ../results/metrics_2_6
+++++++++++ [2_7] | 1 ++++++++++++++++++
Elapsed time:  0.3455423840000549
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_7] | 2 ++++++++++++++++++
Elapsed time:  0.290193535999947
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_7] | 3 ++++++++++++++++++
Elapsed time:  0.24932298600003833
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_7] | 4 ++++++++++++++++++
Elapsed time:  0.27406910100000914
Stopped at epoch:  7
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_7] | 5 ++++++++++++++++++
Elapsed time:  0.4353950720000057
Stopped at epoch:  11
1/1 [==============================] - 0s 15ms/step
+++++++++++ [2_7] | 6 ++++++++++++++++++
Elapsed time:  0.24287572299999738
Stopped at epoch:  6
1/1 [==============================] - 0s 18ms/step
+++++++++++ [2_7] | 7 ++++++++++++++++++
Elapsed time:  0.282758674

...

DataFrame salvo em ../results/metrics_6_3
+++++++++++ [6_4] | 1 ++++++++++++++++++
Elapsed time:  0.32492074700007834
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_4] | 2 ++++++++++++++++++
Elapsed time:  0.48907665100000486
Stopped at epoch:  9
1/1 [==============================] - 0s 23ms/step
+++++++++++ [6_4] | 3 ++++++++++++++++++
Elapsed time:  0.3510346400000799
Stopped at epoch:  7
1/1 [==============================] - 0s 23ms/step
+++++++++++ [6_4] | 4 ++++++++++++++++++
Elapsed time:  0.32121949500015035
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_4] | 5 ++++++++++++++++++
Elapsed time:  0.3732240889999048
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [6_4] | 6 ++++++++++++++++++
Elapsed time:  0.3883568080000259
Stopped at epoch:  8
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_4] | 7 ++++++++++++++++++
Elapsed time:  0.323266326

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_6_4_0,0.50004,-0.460491,0.807888,0.625447,60.295319,58.026169,29.474684,44.590179


DataFrame salvo em ../results/metrics_6_4
+++++++++++ [6_5] | 1 ++++++++++++++++++
Elapsed time:  0.27502003299991884
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_5] | 2 ++++++++++++++++++
Elapsed time:  0.28987839800015536
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [6_5] | 3 ++++++++++++++++++
Elapsed time:  0.3869944170000963
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
+++++++++++ [6_5] | 4 ++++++++++++++++++
Elapsed time:  0.6274733800000831
Stopped at epoch:  12
1/1 [==============================] - 0s 18ms/step
+++++++++++ [6_5] | 5 ++++++++++++++++++
Elapsed time:  0.2937564169999405
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_5] | 6 ++++++++++++++++++
Elapsed time:  0.2907645410000441
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [6_5] | 7 ++++++++++++++++++
Elapsed time:  0.356555731

,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt
model_6_9_8,0.517319,-0.336642,-0.292062,-0.011280,58.211506,88.471413,199.881744,140.899536
model_6_9_0,0.517391,-0.263319,-0.326434,-0.010792,58.202740,83.618248,205.199051,140.831482
model_6_9_11,0.517729,-0.340721,-0.287910,-0.010141,58.162079,88.741432,199.239441,140.740753
model_6_9_4,0.517786,-0.313196,-0.300704,-0.009896,58.155159,86.919533,201.218643,140.706650
model_6_9_12,0.518078,-0.338445,-0.287099,-0.009153,58.119934,88.590790,199.113998,140.603119
model_6_9_5,0.518122,-0.310142,-0.300340,-0.008943,58.114613,86.717438,201.162292,140.573837
model_6_9_10,0.518167,-0.334597,-0.288476,-0.008887,58.109215,88.336044,199.326981,140.566040
model_6_9_7,0.518284,-0.326851,-0.291472,-0.008515,58.095066,87.823349,199.790482,140.514236
model_6_9_1,0.518375,-0.272023,-0.317001,-0.008059,58.084126,84.194374,203.739731,140.450668
model_6_9_6,0.518861,-0.311139,-0.295851,-0.006851,58.025517,86.783379,200.467926,140.282394


DataFrame salvo em ../results/metrics_6_9
